In [1]:
!wget -O dataset.txt 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt'

--2023-06-01 22:51:19--  https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 20623547 (20M) [text/plain]
저장 위치: `dataset.txt'

dataset.txt         100%[===================>]  19.67M  1021KB/s    /  18s     

2023-06-01 22:51:39 (1.10 MB/s) - `dataset.txt' 저장함 [20623547/20623547]



In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
raw = pd.read_table('./dataset.txt', names=['rating', 'review'])

raw['label'] = np.where( raw['rating'] > 3, 1, 0 )
print(raw)

        rating                                             review  label
0            5                                            배공빠르고 굿      1
1            2                      택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고      0
2            5  아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...      1
3            2  선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...      0
4            5                  민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ      1
...        ...                                                ...    ...
199995       2                                    장마라그런가!!! 달지않아요      0
199996       5  다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...      1
199997       5                    로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요      1
199998       5                                      넘이쁘고 쎄련되보이네요~      1
199999       5   아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다      1

[200000 rows x 3 columns]


In [3]:
# 데이터전처리
raw['review'] = raw['review'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]', '')

# print(raw.isnull().sum) # null값 있나 확인
raw.drop_duplicates( subset=['review'], inplace=True ) # 중복자료 제거

In [4]:
# bag of words
uniqueSy = raw['review'].tolist()
uniqueSy = ''.join(uniqueSy)
uniqueSy = list(set(uniqueSy))
uniqueSy.sort()

print(uniqueSy[0:100])

['\x02', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '¡', '£', '¥', '¨']


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(char_level=True, oov_token='<OOV>') # OOV Out Of Vocabulary

charList = raw['review'].tolist()
tokenizer.fit_on_texts(charList)

print(tokenizer.word_index)

{'<OOV>': 1, ' ': 2, '요': 3, '.': 4, '이': 5, '고': 6, '다': 7, '아': 8, '는': 9, '어': 10, '하': 11, '니': 12, '가': 13, '서': 14, '도': 15, '지': 16, '네': 17, '에': 18, '데': 19, '좋': 20, '구': 21, '사': 22, '기': 23, '해': 24, '은': 25, '로': 26, '만': 27, '나': 28, '무': 29, '보': 30, '있': 31, '매': 32, '게': 33, '~': 34, '리': 35, '그': 36, '잘': 37, '안': 38, '한': 39, '주': 40, '라': 41, '배': 42, '거': 43, '너': 44, '했': 45, '습': 46, '시': 47, '송': 48, '용': 49, '품': 50, '으': 51, '제': 52, '!': 53, '같': 54, '을': 55, '입': 56, '상': 57, 'ㅠ': 58, '들': 59, '인': 60, '자': 61, '면': 62, '재': 63, '스': 64, '정': 65, '먹': 66, '비': 67, '합': 68, '부': 69, '대': 70, '없': 71, '여': 72, '려': 73, '마': 74, '음': 75, '맛': 76, '^': 77, '되': 78, '전': 79, '더': 80, '장': 81, '쓰': 82, '문': 83, '일': 84, '것': 85, 'ㅎ': 86, '건': 87, '저': 88, '않': 89, '수': 90, '생': 91, '르': 92, '빠': 93, '세': 94, '분': 95, '편': 96, '진': 97, '조': 98, '오': 99, '감': 100, '았': 101, '물': 102, '번': 103, '드': 104, '었': 105, '할': 106, '개': 107, '많': 108, '각': 109, '냥': 110, '겠'

In [6]:
raw['length'] = raw['review'].str.len()
# print(raw.head())
print(raw.describe())

# raw['length'][raw['length'] < 100].count()

              rating          label         length
count  199908.000000  199908.000000  199908.000000
mean        3.225909       0.499995      39.699402
std         1.645431       0.500001      29.168119
min         1.000000       0.000000       3.000000
25%         2.000000       0.000000      17.000000
50%         2.000000       0.000000      29.000000
75%         5.000000       1.000000      55.000000
max         5.000000       1.000000     140.000000


In [7]:
train_seq = tokenizer.texts_to_sequences(charList)
validate_seq = raw['label'].tolist()

In [8]:
# 패딩맞추기 (문장길이 동일하게 맞춰주기)
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split # 데이터 쪼개서 훈련용 검증용 나눔

X = pad_sequences(train_seq, maxlen=120)
trainX, valX, trainY, valY = train_test_split(X, validate_seq, test_size=0.2, random_state=42)


In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding( len(uniqueSy) + 1, 16), # 원핫인코딩 대신 (데이터 백터화)
  tf.keras.layers.LSTM( 120 ),
  tf.keras.layers.Dense( 1, activation='softmax') 
])

model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-06-02 09:55:42.236724: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-02 09:55:42.236746: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          57056     
                                                                 
 lstm (LSTM)                 (None, 120)               65760     
                                                                 
 dense (Dense)               (None, 1)                 121       
                                                                 
Total params: 122937 (480.22 KB)
Trainable params: 122937 (480.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
saveModel = tf.keras.callbacks.ModelCheckpoint(
  filepath='../../models/checkpoints/RNN_NLPComment/',
  monitor='val_acc',
  mode='max',
  save_freq='epoch',
)

model.fit(np.asarray(trainX), np.asarray(trainY), epochs=500, verbose=1, callbacks=[saveModel])
model.save('../../models/RNN_NLPComment/')

Epoch 1/500
4998/4998 [==============================] - ETA: 0s - loss: 0.2700INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_NLPComment/assets


INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_NLPComment/assets


4998/4998 [==============================] - 126s 25ms/step - loss: 0.2700
Epoch 2/500
4998/4998 [==============================] - ETA: 0s - loss: 0.2373INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_NLPComment/assets


INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_NLPComment/assets


4998/4998 [==============================] - 124s 25ms/step - loss: 0.2373
Epoch 3/500
4997/4998 [============================>.] - ETA: 0s - loss: 0.2205INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_NLPComment/assets


...

4998/4998 [==============================] - 122s 24ms/step - loss: 0.0149
Epoch 236/500
4997/4998 [============================>.] - ETA: 0s - loss: 0.0133INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_NLPComment/assets


INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_NLPComment/assets


4998/4998 [==============================] - 123s 25ms/step - loss: 0.0133
Epoch 237/500
4998/4998 [==============================] - ETA: 0s - loss: 0.0135INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_NLPComment/assets


INFO:tensorflow:Assets written to: ../../models/checkpoints/RNN_NLPComment/assets


4998/4998 [==============================] - 124s 25ms/step - loss: 0.0135
Epoch 238/500
4570/4998 [==========================>...] - ETA: 10s - loss: 0.0123

In [10]:
model = tf.keras.models.load_model('../../models/checkpoints/RNN_NLPComment')
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, None, 16)          57056     
                                                                 
 lstm_9 (LSTM)               (None, 120)               65760     
                                                                 
 dense_9 (Dense)             (None, 1)                 121       
                                                                 
Total params: 122937 (480.22 KB)
Trainable params: 122937 (480.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
score = model.evaluate(np.asarray(valX), np.asarray(valY))
print(score)

   1/1250 [..............................] - ETA: 6:17 - loss: 0.8305

2023-06-02 09:56:56.027468: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-02 09:56:56.096431: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1250/1250 [==============================] - 11s 8ms/step - loss: 0.7542
0.7541609406471252
